In [1]:
from selenium import webdriver
from selenium.webdriver.support.ui import Select
import pandas as pd
import time

In [2]:
chromedriver_location = "chrome_driver/chromedriver"

In [3]:
# xPaths to each required search element in the field. 
district_drop_down = '//*[@id="courtId"]'
date_field = '//*[@id="inputVO.startDate"]'
seven_day_search_radio = '//*[@id="sevenDayRadio"]'
search_by_attorney_radio = '//*[@id="searchAttorneyRadio"]'
mobar_input = '//*[@id="inputVO.mobarNumber"]'
submit_button = '//*[@id="findButton"]'

In [4]:
# create an instance of the webdriver and load the inital calendar search page
driver = webdriver.Chrome(chromedriver_location)
driver.get('https://www.courts.mo.gov/casenet/cases/calendarSearch.do')

# store the district dropdown in an object using Select
courtDistrictDrp = Select(driver.find_element_by_xpath(district_drop_down))
courtDistrictDrp.select_by_value("CT11") # 11th District Saint Charles

# date
driver.find_element_by_xpath(date_field).send_keys("8/10/2021")
# 7 day search results
driver.find_element_by_xpath(seven_day_search_radio).click()
# search by attorney
driver.find_element_by_xpath(search_by_attorney_radio).click()
# MOBAR number field 
driver.find_element_by_xpath(mobar_input).send_keys("71855")

# without the pause, the inputs are too fast for casenet to handle
time.sleep(1) 

# submit 
driver.find_element_by_xpath(submit_button).click()



In [40]:
dates = driver.find_elements_by_xpath('//td[contains(text(), ", 20")]')

In [41]:
for h in dates:
    print(h.text)

August 11, 2021 - August 17, 2021
 Wednesday, August 11, 2021
 Thursday, August 12, 2021


In [5]:
i=1
data = []
while driver.find_elements_by_xpath(f'//td[@class="td{i}"]'):
    data.append(driver.find_elements_by_xpath(f'//td[@class="td{i}"]'))
    i += 1

In [17]:
raw_text = []
for d in data:
    for i in d:
        raw_text.append(i.text)

In [24]:
def slice_per(source, step):
    return [source[i::step] for i in range(step)]

In [27]:
organized_data = slice_per(raw_text, 9)

In [28]:
len(organized_data)

9

In [29]:
organized_data

[['2111-CC00542  ',
  '1611-CR03019-01  ',
  '1511-FC01981-01  ',
  '1711-CR02411-01  '],
 ['STEPHEN WYATT V CHESTERFIELD MUNICIPAL ET AL ',
  'ST V MARCOS ENRICO TRUJILLO ',
  'MELISSA Y HARPER V DEREK L HARPER ',
  'ST V JEREMY LEE BRANDT '],
 ['9:00 AM ', '9:00 AM ', '9:00 AM ', '9:00 AM '],
 ['1 OF 1', '1 OF 1', '1 OF 1', '1 OF 1'],
 [' ', ' ', ' ', ' '],
 ['Civil Setting ',
  'Probation Viol Hearing ',
  'Case Review ',
  'Disposition Hearing '],
 ['Event Text:  ',
  'Event Text:  ',
  'Event Text: SUBMISSION OF DOCUMENTS PROPOSED CUSTODY SCHEDULE',
  'Event Text: '],
 ['Location: DIVISION 1 COURTROOM St Charles Circuit Div',
  'Location: DIVISION 5 COURTROOM St Charles Circuit Div',
  'Location: DIVISION 3 COURTROOM St Charles Circuit Div',
  'Location: DIVISION 5 COURTROOM St Charles Circuit Div'],
 ['Judge/Commissioner: REBECA NAVARRO-MCKELVEY ',
  'Judge/Commissioner: JON A. CUNNINGHAM ',
  'Judge/Commissioner: JOHN PAUL BANAS',
  'Judge/Commissioner: JON A. CUNNINGHAM']]